In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import os


2025-10-20 15:08:37.923471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760972918.159498      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760972918.224248      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
path='/kaggle/input/fer-and-raf-db/datset'
train=os.path.join(path,'train')
test=os.path.join(path,'test')

In [3]:
emotions=['angry', 'happy', 'sad', 'surprise', 'neutral']

In [4]:
train_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    rotation_range=25,
    width_shift_range=0.25,
    height_shift_range=0.25,
    zoom_range=0.25,
    brightness_range=[0.6, 1.4],
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
    
)

train_data = train_gen.flow_from_directory(
    train,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=emotions
)


val_data = train_gen.flow_from_directory(
    train,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=emotions
)


test_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

test_data = test_gen.flow_from_directory(
    test,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    classes=emotions
)



Found 28361 images belonging to 5 classes.
Found 7088 images belonging to 5 classes.
Found 8877 images belonging to 5 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

def transformer_encoder(inputs, num_heads=4, key_dim=64, ff_dim=256, dropout=0.1):
    # Multi-Head Self Attention
    attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(inputs, inputs)
    attention = layers.Dropout(dropout)(attention)
    attention = layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    # Feed Forward
    ff = layers.Dense(ff_dim, activation="relu")(attention)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.Dropout(dropout)(ff)
    return layers.LayerNormalization(epsilon=1e-6)(attention + ff)


def build_vgg16_transformer(input_shape=(128, 128, 3), num_classes=5):
    
    base_model = tf.keras.applications.VGG16(
        input_shape=input_shape,
        include_top=False,
        weights="imagenet"
    )
    
    

    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs)  

    
    feature_dim = x.shape[-1]
    x = layers.Reshape((-1, feature_dim))(x) 

    for _ in range(2):
        x = transformer_encoder(x, num_heads=8, ff_dim=512)
    x = transformer_encoder(x, num_heads=4, ff_dim=256)
    x = transformer_encoder(x, num_heads=4, ff_dim=256)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128)(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    
    model = models.Model(inputs, outputs)
    return model

# Build model
model = build_vgg16_transformer()
model.summary()

I0000 00:00:1760972955.777033      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1760972955.777795      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 4, 4, 512) │ 14,714,688 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 16, 512)   │          0 │ vgg16[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 16, 512)   │  1,050,624 │ reshape[0][0],    │
│ (MultiHeadAttentio… │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16, 512)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 16, 512)   │          0 │ reshape[0][0],    │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 16, 512)   │      1,024 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16, 512)   │    262,656 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16, 512)   │    262,656 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 16, 512)   │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 16, 512)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 16, 512)   │      1,024 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 16, 512)   │  1,050,624 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 16, 512)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 16, 512)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 16, 512)   │      1,024 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16, 512)   │    262,656 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16, 512)   │    262,656 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 16, 512)   │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 16, 512)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0] 

 Total params: 19,518,021 (74.46 MB)

 Trainable params: 19,518,021 (74.46 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=loss, metrics=['accuracy'])


In [7]:
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.callbacks import ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=7, restore_best_weights=True
)
model.fit(train_data, validation_data=val_data, epochs=60, callbacks=[reduce_lr, early_stop])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60


I0000 00:00:1760972980.457843      84 service.cc:148] XLA service 0x7c3690003850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760972980.458825      84 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760972980.458851      84 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1760972982.684367      84 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1760973008.989667      84 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


887/887 ━━━━━━━━━━━━━━━━━━━━ 425s 422ms/step - accuracy: 0.3027 - loss: 1.6945 - val_accuracy: 0.5102 - val_loss: 1.2858 - learning_rate: 1.0000e-04
Epoch 2/60
887/887 ━━━━━━━━━━━━━━━━━━━━ 274s 308ms/step - accuracy: 0.5197 - loss: 1.2797 - val_accuracy: 0.5926 - val_loss: 1.1398 - learning_rate: 1.0000e-04
Epoch 3/60
887/887 ━━━━━━━━━━━━━━━━━━━━ 254s 286ms/step - accuracy: 0.6169 - loss: 1.1139 - val_accuracy: 0.6514 - val_loss: 1.0551 - learning_rate: 1.0000e-04
Epoch 4/60
887/887 ━━━━━━━━━━━━━━━━━━━━ 284s 320ms/step - accuracy: 0.6731 - loss: 1.0323 - val_accuracy: 0.6505 - val_loss: 1.0579 - learning_rate: 1.0000e-04
Epoch 5/60
887/887 ━━━━━━━━━━━━━━━━━━━━ 345s 389ms/step - accuracy: 0.6928 - loss: 0.9920 - val_accuracy: 0.6848 - val_loss: 1.0101 - learning_rate: 1.0000e-04
Epoch 6/60
887/887 ━━━━━━━━━━━━━━━━━━━━ 278s 314ms/step - accuracy: 0.7203 - loss: 0.9428 - val_accuracy: 0.6579 - val_loss: 1.0430 - learning_rate: 1.0000e-04
Epoch 7/60
887/887 ━━━━━━━━━━━━━━━━━━━━ 268s 302ms/

In [8]:
pred=model.predict(test_data)

278/278 ━━━━━━━━━━━━━━━━━━━━ 70s 246ms/step


In [9]:
print(accuracy_score(test_data.labels, np.argmax(pred, axis=1)))

0.7997071082572942


In [10]:
y_pred_classes = np.argmax(pred, axis=1)


y_true = test_data.classes
class_labels = list(test_data.class_indices.keys())

In [11]:
from sklearn.metrics import classification_report

report = classification_report(
    y_true, 
    y_pred_classes, 
    target_names=class_labels
)
print(report)

              precision    recall  f1-score   support

       angry       0.73      0.70      0.71      1120
       happy       0.91      0.92      0.91      2959
         sad       0.71      0.68      0.70      1725
    surprise       0.86      0.86      0.86      1160
     neutral       0.71      0.75      0.73      1913

    accuracy                           0.80      8877
   macro avg       0.78      0.78      0.78      8877
weighted avg       0.80      0.80      0.80      8877



In [12]:
model.save('vgg16_transformer.keras')